In [1]:
import time
import json
import string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud


# download stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

/home/awmarcel/miniconda3/envs/torch-env/lib/python3.10/site-packages/numpy/_core/getlimits.py:545: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)
[nltk_data] Downloading package punkt to /home/awmarcel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/awmarcel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/awmarcel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# Functions
def convert_json_to_pandas(file_path: str) -> pd.DataFrame:
    with open(file_path, 'r') as f:
        data = json.load(f) 
        return pd.json_normalize(data)

In [3]:
# Load data:
df = convert_json_to_pandas('../data/raw/complaints-2021-05-14_08_16_.json')

# Rename columns:
df.columns = [
    'index', 'type', 'id', 'score', 'tags', 'zip_code',
    'complaint_id', 'issue', 'date_received', 'state', 
    'consumer_disputed', 'product', 'company_response', 
    'company', 'submitted_via', 'date_sent_to_company', 
    'company_public_response', 'sub_product', 'timely',
    'complaint_what_happened', 'sub_issue', 'consumer_consent_provided'
]

# Handle missing values:
df['complaint_what_happened'] = df['complaint_what_happened'].fillna(np.nan)
df = df[df.consumer_consent_provided == 'Consent provided'].reset_index(drop=True)

In [4]:
lemmatizer = WordNetLemmatizer()

# Function:
def preprocess_text(text: str, stop_words: list, punctuations: list) -> str:
    # Tokenize text:
    tokens = word_tokenize(text.lower())
    
    # Remove stop words and punctuations:
    filtered_tokens = [
        token
        for token in tokens
        if token not in stop_words and token not in punctuations
    ]  
    
    # Lemmatization:
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]
    return " ".join(lemmatized_tokens)


In [6]:
# Prepare texts, stop words and punctuations
texts = df['complaint_what_happened'].tolist()
stop_words = stopwords.words("english")
punctuations = string.punctuation

# Text preprocessing:
start = time.perf_counter()
cleaned_texts = [preprocess_text(text, stop_words, punctuations) for text in texts]
end = time.perf_counter()
print(f"Text preprocessing took: {end-start:.1f} s")

Text preprocessing took: 40.0 s


In [ ]:
# Create and generate a word cloud image:
wordcloud = WordCloud().generate(str(texts))

# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.title("Word Cloud")
plt.axis("off")
plt.show()